# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686614


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:18,  2.80s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:01,  2.27s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:40,  1.61s/it]

Rendering models:  23%|██▎       | 7/30 [00:06<00:26,  1.14s/it]

...

Rendering models:  70%|███████   | 21/30 [00:07<00:01,  5.89it/s]

Rendering models:  77%|███████▋  | 23/30 [00:08<00:00,  7.00it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  8.55it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00, 10.51it/s]

Petrusperes                           0.001096
CThomas                               0.001560
not-logged-in-0b3c4d412fae103c1ef6    0.005342
Camelron                              0.000260
OjO                                   0.001364
not-logged-in-87d7b7ba881f074d8468    0.014170
mtajnai                               0.000336
bldelacr                              0.000367
jmfranci                              0.001533
justsomenoodles                       0.000388
not-logged-in-923f38b394a04ae3a577    0.033597
not-logged-in-5f30b2bb9b7f9d49da41    0.001918
databanana                            0.000853
aidenr                                0.001179
Lavadude                              0.011489
not-logged-in-15498ebaa693dd5a198d    0.000276
not-logged-in-8700ff8251e0704f745c    0.007313
ShirleyL                              0.000364
bboyle23                              0.001949
vceccare                              0.000770
Getrekt10127                          0.012350
Mkcls        

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())